In [14]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [15]:
print('This notebook was run on ' + time.strftime("%d/%m/%Y") + ' at ' + time.strftime("%H:%M:%S") + '.')

This notebook was run on 06/11/2023 at 22:13:14.


In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [17]:
dataset_directory = os.path.join(os.getenv('HOME'), 'icar-ng-data', 'dataset')
print('Dataset directory: {}'.format(dataset_directory))

dataset_file = 'dataset.csv'
print('Dataset file: {}'.format(dataset_file))

dataset_path = os.path.join(dataset_directory, dataset_file)
print('Dataset path: {}'.format(dataset_path))

# ======================================

model_directory = os.path.join(os.getenv('HOME'), 'icar-ng-data', 'model')
print('Model directory: {}'.format(model_directory))

model_file = 'cm2pixel_model.pt'
print('Model file: {}'.format(model_file))

model_path = os.path.join(model_directory, model_file)
print('Model path: {}'.format(model_path))

Dataset directory: /home/icar/icar-ng-data/dataset
Dataset file: dataset.csv
Dataset path: /home/icar/icar-ng-data/dataset/dataset.csv
Model directory: /home/icar/icar-ng-data/model
Model file: cm2pixel_model.pt
Model path: /home/icar/icar-ng-data/model/cm2pixel_model.pt


In [18]:
# Check whether dataset file exists
if not os.path.exists(dataset_path):
    print('Dataset file does not exist: {}'.format(dataset_path))
    exit()

# Check whether model directory exists
if not os.path.exists(model_directory):
    os.makedirs(model_directory)
    print('Created model directory: {}'.format(model_directory))

In [19]:
df_raw = pd.read_csv(dataset_path)
df_train = df_raw.sample(frac=0.8)
df_test = df_raw.drop(df_train.index)

min_x = np.min(df_train.iloc[:, 2:4].values, axis=0)
min_x = torch.tensor(min_x, dtype=torch.float32)
max_x = np.max(df_train.iloc[:, 2:4].values, axis=0)
max_x = torch.tensor(max_x, dtype=torch.float32)
min_y = np.min(df_train.iloc[:, 0:2].values, axis=0)
min_y = torch.tensor(min_y, dtype=torch.float32)
max_y = np.max(df_train.iloc[:, 0:2].values, axis=0)
max_y = torch.tensor(max_y, dtype=torch.float32)

print('min_x: {}, max_x: {}'.format(min_x, max_x))
print('min_y: {}, max_y: {}'.format(min_y, max_y))

min_x: tensor([   0.0000, -848.5281]), max_x: tensor([1183.6727,  848.5281])
min_y: tensor([ 30., 204.]), max_y: tensor([1268.,  692.])


In [20]:
class ForwardDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        x = self.dataframe.iloc[:, 2:4].values
        y = self.dataframe.iloc[:, 0:2].values
        self.x = torch.from_numpy(x).float()
        self.y = torch.from_numpy(y).float()
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [21]:
dataset_train = ForwardDataset(df_train)
dataset_test = ForwardDataset(df_test)
print('dataset_train: {}'.format(len(dataset_train)))
print('dataset_test: {}'.format(len(dataset_test)))

dataloader_train = DataLoader(dataset_train, batch_size=64, shuffle=True, pin_memory=True)
dataloader_test = DataLoader(dataset_test, batch_size=64, shuffle=True, pin_memory=True)

dataset_train: 118
dataset_test: 29


In [22]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_size, output_size, min_x, max_x, min_y, max_y):
        super(MultiLayerPerceptron, self).__init__()
        self.min_x = min_x
        self.max_x = max_x
        self.min_y = min_y
        self.max_y = max_y
        self.fc1 = nn.Linear(input_size, 4)
        self.fc2 = nn.Linear(4, 20)
        self.fc3 = nn.Linear(20, 80)
        self.fc4 = nn.Linear(80, 20)
        self.fc5 = nn.Linear(20, 4)
        self.fc6 = nn.Linear(4, output_size)

    def forward(self, x):
        x = (x - self.min_x) / (self.max_x - self.min_x)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        x = self.fc6(x)
        x = x * (self.max_y - self.min_y) + self.min_y
        return x

In [23]:
model = MultiLayerPerceptron(2, 2, min_x.to(device), max_x.to(device), min_y.to(device), max_y.to(device)).to(device)

if os.path.exists(model_path):
    try:
        print('Loading model: {}'.format(model_path))
        model.load_state_dict(torch.load(model_path))
        print('Loaded model: {}'.format(model_path))
    except BaseException as e:
        print('Failed to load model: {}'.format(model_path))
        print(e)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

Loading model: /home/icar/icar-ng-data/model/cm2pixel_model.pt
Loaded model: /home/icar/icar-ng-data/model/cm2pixel_model.pt


In [24]:
min_test_loss = np.inf

for epoch in range(50000):
    model.train()
    train_loss = 0.0
    for i, (x, y) in enumerate(dataloader_train):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    if epoch % 100 == 0:
        model.eval()
        test_loss = 0.0
        for i, (x, y) in enumerate(dataloader_test):
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            test_loss += loss.item()

        save_model = False
        if test_loss < min_test_loss:
            min_test_loss = test_loss
            torch.save(model.state_dict(), model_path)
            save_model = True

        if save_model:
            print('epoch: {}, train_loss: {:.6f}, test_loss: {:.6f} (Saved)'.format(epoch, train_loss, test_loss))
        else:
            print('epoch: {}, train_loss: {:.6f}, test_loss: {:.6f}'.format(epoch, train_loss, test_loss))

epoch: 0, train_loss: 134501.785156, test_loss: 55761.058594 (Saved)
epoch: 100, train_loss: 35172.310547, test_loss: 13474.715820 (Saved)
epoch: 200, train_loss: 8552.485596, test_loss: 3704.319336 (Saved)
epoch: 300, train_loss: 3103.734741, test_loss: 1667.260132 (Saved)
epoch: 400, train_loss: 1375.375305, test_loss: 800.460388 (Saved)
epoch: 500, train_loss: 673.516144, test_loss: 380.012024 (Saved)
epoch: 600, train_loss: 382.810699, test_loss: 213.968170 (Saved)
epoch: 700, train_loss: 273.191597, test_loss: 150.821182 (Saved)
epoch: 800, train_loss: 229.603897, test_loss: 124.082581 (Saved)
epoch: 900, train_loss: 203.173988, test_loss: 108.347488 (Saved)
epoch: 1000, train_loss: 181.315079, test_loss: 96.464447 (Saved)
epoch: 1100, train_loss: 168.681999, test_loss: 86.958275 (Saved)
epoch: 1200, train_loss: 153.439651, test_loss: 79.178795 (Saved)
epoch: 1300, train_loss: 141.075928, test_loss: 72.659805 (Saved)
epoch: 1400, train_loss: 129.835388, test_loss: 67.258720 (Saved

In [25]:
onnx_path = os.path.join(model_directory, 'cm2pixel_model.onnx')
print('ONNX path: {}'.format(onnx_path))

try:
    print('Saving ONNX model: {}'.format(onnx_path))
    torch.onnx.export(model, torch.randn(1, 2).to(device), onnx_path, verbose=True)
    print('Saved ONNX model: {}'.format(onnx_path))
except BaseException as e:
    print('Failed to save ONNX model: {}'.format(onnx_path))
    print(e)

ONNX path: /home/icar/icar-ng-data/model/cm2pixel_model.onnx
Saving ONNX model: /home/icar/icar-ng-data/model/cm2pixel_model.onnx
Exported graph: graph(%onnx::Sub_0 : Float(1, 2, strides=[2, 1], requires_grad=0, device=cuda:0),
      %fc1.weight : Float(4, 2, strides=[2, 1], requires_grad=1, device=cuda:0),
      %fc1.bias : Float(4, strides=[1], requires_grad=1, device=cuda:0),
      %fc2.weight : Float(20, 4, strides=[4, 1], requires_grad=1, device=cuda:0),
      %fc2.bias : Float(20, strides=[1], requires_grad=1, device=cuda:0),
      %fc3.weight : Float(80, 20, strides=[20, 1], requires_grad=1, device=cuda:0),
      %fc3.bias : Float(80, strides=[1], requires_grad=1, device=cuda:0),
      %fc4.weight : Float(20, 80, strides=[80, 1], requires_grad=1, device=cuda:0),
      %fc4.bias : Float(20, strides=[1], requires_grad=1, device=cuda:0),
      %fc5.weight : Float(4, 20, strides=[20, 1], requires_grad=1, device=cuda:0),
      %fc5.bias : Float(4, strides=[1], requires_grad=1, device

In [26]:
model.eval()
for i, (x, y) in enumerate(dataloader_test):
    x = x.to(device)
    y = y.to(device)
    y_pred = model(x)
    
    print('x\n{}'.format(x))
    print('y\n{}'.format(y))
    print('y_pred\n{}'.format(y_pred))
    print('y - y_pred\n{}'.format(y - y_pred))
    
    break
     

x
tensor([[ 350.0000,  100.0000],
        [ 998.4603,  665.6403],
        [ 100.0000,  175.0000],
        [ 250.0000, -100.0000],
        [1200.0000,    0.0000],
        [  50.0000,   50.0000],
        [ 350.0000, -200.0000],
        [ 200.0000, -300.0000],
        [   0.0000,  -75.0000],
        [  75.0000,   50.0000],
        [ 100.0000, -125.0000],
        [ 250.0000, -300.0000],
        [ 416.0251,  277.3501],
        [ 350.0000, -100.0000],
        [ 447.2136, -223.6068],
        [ 715.5417, -357.7709],
        [ 100.0000, -175.0000],
        [ 350.0000,  150.0000],
        [ 200.0000,  150.0000],
        [ 350.0000,  -50.0000],
        [ 250.0000,   50.0000],
        [  50.0000, -100.0000],
        [ 250.0000,  250.0000],
        [ 150.0000,  250.0000],
        [ 998.4603, -665.6403],
        [ 450.0000,    0.0000],
        [ 100.0000,    0.0000],
        [ 474.3416,  158.1139],
        [ 450.0000, -200.0000]], device='cuda:0')
y
tensor([[ 500.,  292.],
        [ 246.,  220.],
  